# Imports

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
spark = (SparkSession
         .builder
         .appName('Chapter 4')
         .enableHiveSupport()
         .getOrCreate())
spark

# SQL Operations

We create schema using DDL-like or StructType kinds of definition.

In [3]:
# DDL
schema = '''
`date` STRING, `delay` INT, `distance` INT, `origin` STRING, `destination` STRING
'''

In [4]:
df = (spark
      .read
      .format('csv')
      .option('header', True)
      .option('schema', schema)
      .load('data/departuredelays.csv'))

In [5]:
df.show(5, truncate=False)

+--------+-----+--------+------+-----------+
|date    |delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|6    |602     |ABE   |ATL        |
|01020600|-8   |369     |ABE   |DTW        |
|01021245|-2   |602     |ABE   |ATL        |
|01020605|-4   |602     |ABE   |ATL        |
|01031245|-4   |602     |ABE   |ATL        |
+--------+-----+--------+------+-----------+
only showing top 5 rows



In [6]:
# Because year is missing, this UDF will convert date into DateType
def to_date_format_udf(d_str):
    l = [char for char in d_str]
    return "".join(l[0:2]) + "/" +  "".join(l[2:4]) + " " + " " +"".join(l[4:6]) + ":" + "".join(l[6:])

to_date_format_udf("02190925")

'02/19  09:25'

In [7]:
spark.udf.register('to_date', to_date_format_udf, StringType())

<function __main__.to_date_format_udf(d_str)>

In [8]:
df.createOrReplaceTempView('us_delay_flights_tbl')

In [9]:
spark.sql('''
SELECT * 
FROM us_delay_flights_tbl
WHERE distance > 1000
ORDER BY distance DESC
''').show(10)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|03131530|    0|    4330|   HNL|        JFK|
|03071625|   -1|    4330|   HNL|        JFK|
|03121530|   -3|    4330|   HNL|        JFK|
|03021625|   14|    4330|   HNL|        JFK|
|03061625|   -2|    4330|   HNL|        JFK|
|03081530|    4|    4330|   HNL|        JFK|
|03091530|   -7|    4330|   HNL|        JFK|
|03011625|   -1|    4330|   HNL|        JFK|
|03151530|    2|    4330|   HNL|        JFK|
|03051625|   -6|    4330|   HNL|        JFK|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [10]:
(df
 .where('distance > 1000')
 .sort('distance', ascending=False)
 .show(10))

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01090900|   -3|    4330|   JFK|        HNL|
|01050900|   98|    4330|   JFK|        HNL|
|01080900|   14|    4330|   JFK|        HNL|
|01020900|    1|    4330|   JFK|        HNL|
|01040900|  111|    4330|   JFK|        HNL|
|01060900|   -2|    4330|   JFK|        HNL|
|01070900|    3|    4330|   JFK|        HNL|
|01010900|    6|    4330|   JFK|        HNL|
|01110900|   -4|    4330|   JFK|        HNL|
|01030900|  784|    4330|   JFK|        HNL|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [11]:
(df
 .where('distance > 1000')
 .sort(F.desc('distance'))
 .show(10))

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|02131625|   -1|    4330|   HNL|        JFK|
|02071625|   16|    4330|   HNL|        JFK|
|02121625|  932|    4330|   HNL|        JFK|
|02021625|   -5|    4330|   HNL|        JFK|
|02061625|   -9|    4330|   HNL|        JFK|
|02081625|   -1|    4330|   HNL|        JFK|
|02091625|   -6|    4330|   HNL|        JFK|
|02011625|   -1|    4330|   HNL|        JFK|
|02151625|   -2|    4330|   HNL|        JFK|
|02051625|   -8|    4330|   HNL|        JFK|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [12]:
spark.sql("""
SELECT * 
FROM us_delay_flights_tbl
WHERE delay > 120
AND origin = 'SFO'
AND destination = 'ORD'
ORDER BY distance DESC
""").show(10)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|03201100|  160|    1604|   SFO|        ORD|
|03311810|  139|    1604|   SFO|        ORD|
|03311405|  196|    1604|   SFO|        ORD|
|03120929|  143|    1604|   SFO|        ORD|
|03141657|  165|    1604|   SFO|        ORD|
|03171251|  151|    1604|   SFO|        ORD|
|03171215|  189|    1604|   SFO|        ORD|
|03260828|  184|    1604|   SFO|        ORD|
|03261106|  173|    1604|   SFO|        ORD|
|03272225|  160|    1604|   SFO|        ORD|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [13]:
(df
 .where((F.col('delay') > 120) & (F.col('origin') == 'SFO') & (F.col('destination') == 'ORD'))
 .sort('distance', ascending=False)
 .show(10))

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|02081410|  181|    1604|   SFO|        ORD|
|02091800|  223|    1604|   SFO|        ORD|
|02092330|  142|    1604|   SFO|        ORD|
|02101800|  171|    1604|   SFO|        ORD|
|02180925|  141|    1604|   SFO|        ORD|
|02190925| 1638|    1604|   SFO|        ORD|
|02271410|  145|    1604|   SFO|        ORD|
|02071333|  182|    1604|   SFO|        ORD|
|02081104|  137|    1604|   SFO|        ORD|
|02091823|  156|    1604|   SFO|        ORD|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [14]:
(df
 .where((F.col('delay') > 120))
 .where(F.col('origin') == 'SFO')
 .where(F.col('destination') == 'ORD')
 .sort('distance', ascending=False)
 .show(10))

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|03201100|  160|    1604|   SFO|        ORD|
|03311810|  139|    1604|   SFO|        ORD|
|03311405|  196|    1604|   SFO|        ORD|
|03120929|  143|    1604|   SFO|        ORD|
|03141657|  165|    1604|   SFO|        ORD|
|03171251|  151|    1604|   SFO|        ORD|
|03171215|  189|    1604|   SFO|        ORD|
|03260828|  184|    1604|   SFO|        ORD|
|03261106|  173|    1604|   SFO|        ORD|
|03272225|  160|    1604|   SFO|        ORD|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [15]:
spark.sql("""
SELECT delay, origin, destination,
       CASE
          WHEN delay > 360 THEN 'Very Long Delays'
          WHEN delay > 120 AND delay < 360 THEN  'Long Delays '
          WHEN delay > 60 AND delay < 120 THEN  'Short Delays'
          WHEN delay > 0 and delay < 60  THEN   'Tolerable Delays'
          WHEN delay = 0 THEN 'No Delays'
          ELSE 'No Delays'
       END AS Flight_Delays
FROM us_delay_flights_tbl
ORDER BY origin, delay DESC
""").show(10, truncate=False)


+-----+------+-----------+----------------+
|delay|origin|destination|Flight_Delays   |
+-----+------+-----------+----------------+
|92   |ABE   |ORD        |Short Delays    |
|91   |ABE   |DTW        |Short Delays    |
|9    |ABE   |ATL        |Tolerable Delays|
|9    |ABE   |ATL        |Tolerable Delays|
|9    |ABE   |ATL        |Tolerable Delays|
|9    |ABE   |ORD        |Tolerable Delays|
|9    |ABE   |ATL        |Tolerable Delays|
|9    |ABE   |ATL        |Tolerable Delays|
|89   |ABE   |DTW        |Short Delays    |
|88   |ABE   |ATL        |Short Delays    |
+-----+------+-----------+----------------+
only showing top 10 rows



In [16]:
(df
 .withColumn('Flight_Delays', F.when(F.col('delay') > 360, 'Very Long Delays')
             .when((F.col('delay') < 360) & (F.col('delay') > 120), 'Long Delays')
             .when((F.col('delay') < 120) & (F.col('delay') > 60), 'Short Delays')
             .when((F.col('delay') < 60) & (F.col('delay') > 0), 'Tolerable Delays')
             .otherwise('No Delays'))
 .sort(F.asc('origin'), F.desc('delay'))
 .show(10))

+--------+-----+--------+------+-----------+----------------+
|    date|delay|distance|origin|destination|   Flight_Delays|
+--------+-----+--------+------+-----------+----------------+
|01290607|   92|     569|   ABE|        ORD|    Short Delays|
|02050600|   91|     369|   ABE|        DTW|    Short Delays|
|02091725|    9|     602|   ABE|        ATL|Tolerable Delays|
|03241725|    9|     602|   ABE|        ATL|Tolerable Delays|
|03090600|    9|     602|   ABE|        ATL|Tolerable Delays|
|01050605|    9|     602|   ABE|        ATL|Tolerable Delays|
|01211219|    9|     569|   ABE|        ORD|Tolerable Delays|
|03271725|    9|     602|   ABE|        ATL|Tolerable Delays|
|01210600|   89|     369|   ABE|        DTW|    Short Delays|
|01051245|   88|     602|   ABE|        ATL|    Short Delays|
+--------+-----+--------+------+-----------+----------------+
only showing top 10 rows



# SQL Tables

In [17]:
# Create a database
spark.sql('DROP DATABASE IF EXISTS learn_spark_db CASCADE')
spark.sql('CREATE DATABASE learn_spark_db')
spark.sql('USE learn_spark_db') # From this point, any commands we issue in our application to create tables 
                                # will result in the tables being created in this database and residing under 
                                # the database name learn_spark_db.
spark.catalog.listDatabases()

[Database(name='default', description='Default Hive database', locationUri='file:/Users/imad/Downloads/bigdata/notebooks/Learning-Spark/spark-warehouse'),
 Database(name='learn_spark_db', description='', locationUri='file:/Users/imad/Documents/courses/data-engineering/big-data/notebooks/Learning-Spark/spark-warehouse/learn_spark_db.db')]

In [18]:
# Create managed table
spark.sql('CREATE TABLE us_delay_flights_tbl(data STRING, delat INT, distance INT, origing STRING, destination STRING)')
spark.catalog.listTables()

[Table(name='us_delay_flights_tbl', database='learn_spark_db', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='us_delay_flights_tbl', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [19]:
# Create managed table using dataframe API
(df
 .write
 .mode('overwrite')
 .saveAsTable('us_delay_flights_tbl'))

In [20]:
spark.catalog.listTables()

[Table(name='us_delay_flights_tbl', database='learn_spark_db', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='us_delay_flights_tbl', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [21]:
spark.catalog.listTables(dbName='default')

[Table(name='departure_delays_window', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='us_delay_flights_tbl', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [22]:
spark.catalog.listTables(dbName='learn_spark_db')

[Table(name='us_delay_flights_tbl', database='learn_spark_db', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='us_delay_flights_tbl', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [23]:
spark.catalog.listColumns('us_delay_flights_tbl')

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

In [28]:
# Create External Tables (Unmanaged)
spark.sql("DROP DATABASE IF EXISTS learn_spark_db CASCADE")
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")
spark.sql('''
CREATE TABLE us_delay_flights_tbl(data STRING, delat INT, distance INT, origing STRING, destination STRING)
USING CSV OPTIONS (path 'data/departuredelays.csv')
''')

DataFrame[]

In [29]:
# (df
#  .write
#  .mode('overwrite')
#  .option('path', 'data/departuredelays.csv') # That is where Spark determines whether to create managed/external tabel
#  .saveAsTable('us_delay_flights_tbl'))

In [30]:
spark.catalog.listDatabases()

[Database(name='default', description='Default Hive database', locationUri='file:/Users/imad/Downloads/bigdata/notebooks/Learning-Spark/spark-warehouse'),
 Database(name='learn_spark_db', description='', locationUri='file:/Users/imad/Documents/courses/data-engineering/big-data/notebooks/Learning-Spark/spark-warehouse/learn_spark_db.db')]

In [31]:
spark.catalog.listTables()

[Table(name='us_delay_flights_tbl', database='learn_spark_db', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='us_delay_flights_tbl', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

If we have data sql tables, we can read them without the need to reading from files.
```Python
us_flights_df = spark.sql("SELECT * FROM us_delay_flights_tbl") 
us_flights_df2 = spark.table("us_delay_flights_tbl")
```

# SQL Views

In addition to creating tables, Spark can create views on top of existing tables. Views can be global (visible across all SparkSessions on a given cluster) or session-scoped (visible only to a single SparkSession), and they are temporary: they disappear after your Spark application terminates.

Create temporary (global) views.
```sql
CREATE OR REPLACE GLOBAL TEMP VIEW us_origin_airport_SFO_global_tmp_view AS 
SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE origin = 'SFO';

CREATE OR REPLACE TEMP VIEW us_origin_airport_JFK_tmp_view AS
SELECT date, delay, origin, destination from us_delay_flights_tbl WHERE origin = 'JFK'
```
You can accomplish the same thing with the DataFrame API as follows:
```python
df_sfo = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")
df_jfk = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'JFK'")
    # Create a temporary and global temporary view
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")
```
Once you’ve created these views, you can issue queries against them just as you would against a table. Keep in mind that when accessing a global temporary view you must use the prefix global_temp.<view_name>, because Spark creates global temporary views in a global temporary database called global_temp. For example:
```sql
SELECT * FROM global_temp.us_origin_airport_SFO_global_tmp_view
```
By contrast, you can access the normal temporary view without the global_temp prefix:
```sql
SELECT * FROM us_origin_airport_JFK_tmp_view
```
```python
spark.read.table("us_origin_airport_JFK_tmp_view")
// Or
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view")
```
You can also drop a view just like you would a table:
```sql
DROP VIEW IF EXISTS us_origin_airport_SFO_global_tmp_view;
DROP VIEW IF EXISTS us_origin_airport_JFK_tmp_view
```
```python
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view") 
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")
```

__The difference between temporary and global temporary views being subtle, it can be a source of mild confusion among developers new to Spark. A temporary view is tied to a single SparkSession within a Spark application. In contrast, a global temporary view is visible across multiple SparkSessions within a Spark application. Yes, you can create multiple SparkSessions within a single Spark application—this can be handy, for example, in cases where you want to access (and combine) data from two different SparkSessions that don’t share the same Hive metastore configurations.__

# Reading Data

The patten to reading data from different data sources:
```python
DataFrameReader.format(args).option("key", "value").schema(args).load()
```
Since we can only get a handle of DataFrameReader using `SparkSession.read`, therefore:
```python
spark.read.format(args).option("key", "value").schema(args).load()
```

__Parquet__ is the recommended data format for Spark due to its efficiency in reading and space and it is a columnar storage --> Helps __Catalyst__ optimizer. Parquet saves the schema as metadata so we don't actually need to provide a schema. It is also the default format so we don't need to provide `format` argument to the DataFrameReader.

`saveAsTable` will persist the data in Hive metastore. If we don't have Hive installed, Spark will create that for us and use the metastore going forward. This command with either create a managed table or external table (if path option is provided).

# Writing Data

```python
(DataFrameWriter
 .format(args)
 .option(args)
 .bucketBy(args)
 .partitionBy(args)
 .save(path)
// Or
DataFrameWriter.format(args).option(args).sortBy(args).saveAsTable(table)
```

# Data Sources

## Parquet

It is recommended to save data in Parquet format due to its I/O optimizations and compressions. Files will be stored in a directory structure that has the metadata (schema, version, path, etc), data files, compressed files, and some status files. We only need to specify schema when reading from stream.

In [32]:
df.write.save('test-df.parquet')

In [33]:
spark.sql('''
CREATE OR REPLACE TEMP VIEW us_delay_flights
USING parquet
OPTIONS (path "test-df.parquet")
''')

DataFrame[]

In [34]:
spark.catalog.listTables()

[Table(name='us_delay_flights_tbl', database='learn_spark_db', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='us_delay_flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='us_delay_flights_tbl', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [35]:
spark.sql('SELECT * FROM us_delay_flights').show(10)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|02151800|  108|     290|   ORD|        MSP|
|02151800|  142|     772|   ORD|        DEN|
|02151303|   16|    1516|   ORD|        LAX|
|02151157|    7|    1316|   ORD|        LAS|
|02151818|   55|    1511|   ORD|        PDX|
|02151033|   12|     873|   ORD|        MCO|
|02150941|    0|    1499|   ORD|        SNA|
|02151320|   17|    1604|   ORD|        SFO|
|02151804|    2|    1497|   ORD|        SAN|
|02152000|   17|     119|   ORD|        GRR|
+--------+-----+--------+------+-----------+
only showing top 10 rows



In [36]:
df.show(10)

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        ATL|
|01050605|    9|     602|   ABE|        ATL|
+--------+-----+--------+------+-----------+
only showing top 10 rows



## JSON

It is another popular format that comes in two modes: 
- single line: Each line denotes a single JSON object
- Multiline: The entire mutliline denotes a single JSON object

We can use the same methods to read and write JSON files but change the format into JSON.

## CSV

Typically the fields are separated by commas; however, we can have the fields separated by other delimiters.
```sql
CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl USING csv
OPTIONS (
    path "path-to-csv-files/*", 
    header "true",
    inferSchema "true",
    mode "FAILFAST"
)
```

## Other Formats

There are other formats supported by Spark such as ORC, Avro (used by Kafka), Images, and Binary files. They all follow the same patterns in terms of reading and writing data.